In [1]:
import pygame

pygame 2.6.0 (SDL 2.28.4, Python 3.11.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
import math
import random
import time

In [3]:
pygame.init()

(5, 0)

In [4]:
width,height = 800,600
top_bar_height = 50

label_font = pygame.font.SysFont("comicsans", 24)

In [5]:
win = pygame.display.set_mode((width,height))
pygame.display.set_caption("Aim Trainer")

In [6]:
inc = 400
ev = pygame.USEREVENT
padding = 30

bg_color = (0, 25, 40)

lives = 10

In [7]:
class Target:
    max = 20
    rate = 0.1
    color = "pink"
    color2 = "red"
    color3 = "orange"
    color4 = "yellow"
    def __init__ (self,x,y):
        self.x = x
        self.y = y
        self.size = 0
        self.grow = True

    def update(self):
        if self.size + self.rate >= self.max:
            self.grow = False
        if self.grow:
            self.size += self.rate
        else:
            self.size -= self.rate

    def draw(self,win):
        pygame.draw.circle(win, self.color, (self.x,self.y), self.size)
        pygame.draw.circle(win, self.color2, (self.x,self.y), self.size * 0.9)
        pygame.draw.circle(win, self.color3, (self.x,self.y), self.size * 0.8)
        pygame.draw.circle(win, self.color4, (self.x,self.y), self.size * 0.7)

    def collide(self, x , y):
        dis = math.sqrt((x - self.x)**2 + (y - self.y)**2)
        return dis <= self.size



In [8]:
def draw(win, targets):
    win.fill(bg_color)
    for target in targets:
        target.draw(win)
    

In [9]:
def format_time(secs):
    mil = math.floor(int(secs * 1000 % 1000)/100)
    seconds = int(round(secs % 60, 1))
    minutes = int(secs//60)

    return f"{minutes:02d}:{seconds:02d}.{mil}"
    

In [10]:
def top_bar(win, el_time, pressed, miss):
    pygame.draw.rect(win,"grey",(0, 0, width, top_bar_height))
    time_label = label_font.render(f"Time: {format_time(el_time)}", 1, "black")

    speed = round(pressed/el_time, 1)
    speed_label = label_font.render(f"Speed:{speed} t/s", 1, "black")
    hits_label = label_font.render(f"Hits:{pressed}", 1, "black")
    lives_label = label_font.render(f"Lives:{lives - miss}", 1, "black")

    win.blit(time_label, (5,5))
    win.blit(speed_label, (200,5))
    win.blit(hits_label, (450,5))
    win.blit(lives_label, (650,5))
    

In [11]:
def end_screen(win,el_time,pressed, clicks):
    win.fill(bg_color)
    time_label = label_font.render(f"Time: {format_time(el_time)}", 1, "white")
    speed = round(pressed/el_time, 1)
    speed_label = label_font.render(f"Speed:{speed} t/s", 1, "white")
    hits_label = label_font.render(f"Hits:{pressed}", 1, "white")
    acc = round(pressed/clicks * 100, 2)
    acc_label = label_font.render(f"Accuracy:{acc}%", 1, "white")

    win.blit(time_label, (get_middle(time_label),100))
    win.blit(speed_label, (get_middle(speed_label),200))
    win.blit(hits_label, (get_middle(hits_label),300))
    win.blit(acc_label, (get_middle(acc_label),400))

    pygame.display.update()
    run = True
    while run:
        for event in pygame.event.get():
            if event.type == pygame.QUIT or event.type == pygame.KEYDOWN:
                quit()
    

In [12]:
def get_middle(surface):
    return width/2 - surface.get_width()/2

In [ ]:
def main():
    run = True
    targets = []
    clock = pygame.time.Clock()
    pressed = 0
    miss = 0
    clicks = 0
    start_time = time.time()
    
    pygame.time.set_timer(ev, inc)
    while run:
        clock.tick(60)
        click = False
        mouse_pos = pygame.mouse.get_pos()
        el_time = time.time() - start_time
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False
                break
            if event.type == ev:
                x = random.randint(padding, width - padding)
                y = random.randint(padding + top_bar_height, height - padding)
                target = Target(x,y)
                targets.append(target)
            if event.type == pygame.MOUSEBUTTONDOWN:
                click = True
                clicks += 1
                
        for target in targets:
            target.update()
            if target.size <= 0:
                targets.remove(target)
                miss += 1

            if click and target.collide(*mouse_pos):
                targets.remove(target)
                pressed += 1

        if miss >= lives:
            end_screen(win, el_time, pressed, clicks)
                

        draw(win, targets)
        top_bar(win, el_time, pressed, miss)
        pygame.display.update()     
                
    pygame.quit()

if __name__ == "__main__":
    main()
